## Sequential API 활용하기 

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [16]:
mnist = keras.datasets.mnist 

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test  = x_train/255.0, x_test/255.0

x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

print(len(x_train), len(x_test))

60000 10000


In [17]:
model = keras.Sequential() 

model.add(keras.layers.Conv2D(32, 3, activation = 'relu'))
model.add(keras.layers.Conv2D(64, 3, activation = 'relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [20]:
# model = keras.Sequential([
#     keras.layers.Conv2D(32, 3, activation='relu'),
#     keras.layers.Conv2D(64, 3, activation='relu'),
#     keras.layers.Flatten(),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10, activation='softmax')
# ])

In [15]:
len(model.layers)

5

In [18]:
model.compile(optimizer = 'adam', 
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs = 5)

model.evaluate(x_test, y_test, verbose = 2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1074 - accuracy: 0.9675
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0343 - accuracy: 0.9893
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0202 - accuracy: 0.9936
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0132 - accuracy: 0.9955
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0082 - accuracy: 0.9974
313/313 - 2s - loss: 0.0604 - accuracy: 0.9853


[0.06040515750646591, 0.9853000044822693]

In [ ]:
model.summary()

## Functional API 활용하기 
### 장점    
* 덜 장황함
* 연결성 그래프를 정의하는 동안 모델 검증
* 기능적 모델은 플로팅 및 검사 가능
* 기능 모델은 직렬화 또는 복제 가능

### 단점 
* 기능적 API는 모델을 계층의 DAG로 취급합니다.<br>
이는 대부분의 딥 러닝 아키텍처에 해당되지만 전부는 아닙니다. 예를 들어 재귀 네트워크 또는 트리 RNN은이 가정을 따르지 않으며 기능 API에서 구현할 수 없습니다.

In [29]:
# End to End 예제
model_input = keras.Input(shape = (28,28,1), name = "input")
models = keras.layers.Conv2D(32, 3, activation = 'relu')(model_input)
models = keras.layers.Conv2D(64, 3, activation = 'relu')(models)
models = keras.layers.Flatten()(models)
models = keras.layers.Dense(128, activation = 'relu')(models)
models = keras.layers.Dense(10, activation = 'softmax')(models)

model = keras.Model(inputs = model_input, outputs = models)

In [27]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten_5 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               4718720   
_________________________________________________________________
dense_11 (Dense)             (None, 10)                1290      
Total params: 4,738,826
Trainable params: 4,738,826
Non-trainable params: 0
_________________________________________________

In [28]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1101 - accuracy: 0.9665
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0376 - accuracy: 0.9882
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0209 - accuracy: 0.9937
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0130 - accuracy: 0.9958
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0086 - accuracy: 0.9970
313/313 - 0s - loss: 0.0504 - accuracy: 0.9887


[0.05044552683830261, 0.9886999726295471]

## Subclassing 활용하기

In [34]:
class SubclassModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation = 'relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(128, activation = 'relu')
        self.d2 = keras.layers.Dense(10, activation = 'softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [35]:
model = SubclassModel()

In [36]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1340 - accuracy: 0.9598
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0422 - accuracy: 0.9872
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0217 - accuracy: 0.9932
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0136 - accuracy: 0.9956
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0092 - accuracy: 0.9969
313/313 - 0s - loss: 0.0647 - accuracy: 0.9832


[0.0646762028336525, 0.9832000136375427]

# CIFAR-100 데이터셋에 적용
## Sequential API 활용하기

In [37]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print(len(x_train), len(x_test))

50000 10000


In [43]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(16, 3, activation = 'relu'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(keras.layers.Conv2D(32, 3, activation = 'relu'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation = 'relu'))
model.add(keras.layers.Dense(100, activation = 'softmax'))

In [44]:
model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 10s 6ms/step - loss: 3.5973 - accuracy: 0.1606
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.8682 - accuracy: 0.2887
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.5638 - accuracy: 0.3538
Epoch 4/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.3564 - accuracy: 0.3957
Epoch 5/5
1563/1563 [==============================] - 4s 2ms/step - loss: 2.1946 - accuracy: 0.4294
313/313 - 0s - loss: 2.5391 - accuracy: 0.3668


[2.5390684604644775, 0.3668000102043152]

### Funtional API 활용하기 

In [50]:
inputs = keras.Input((32, 32, 3))

models = keras.layers.Conv2D(16, 3, activation = 'relu')(inputs)
models = keras.layers.MaxPool2D(pool_size = (2,2))(models)
models = keras.layers.Conv2D(32, 3, activation = 'relu')(models)
models = keras.layers.MaxPool2D(pool_size = (2,2))(models)
models = keras.layers.Flatten()(models)
models = keras.layers.Dense(256, activation = 'relu')(models)
models = keras.layers.Dense(100, activation = 'softmax')(models)

model = keras.Model(inputs = inputs, outputs = models)

In [51]:
model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 2ms/step - loss: 3.6558 - accuracy: 0.1504
Epoch 2/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.9332 - accuracy: 0.2780
Epoch 3/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.6336 - accuracy: 0.3383
Epoch 4/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.4266 - accuracy: 0.3786
Epoch 5/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.2662 - accuracy: 0.4153
313/313 - 0s - loss: 2.5783 - accuracy: 0.3510


[2.578341007232666, 0.35100001096725464]

In [55]:
class SubclassModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation = 'relu')
        self.maxpool = keras.layers.MaxPool2D(pool_size = (2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation = 'relu')
        self.flat = keras.layers.Flatten() 
        self.d1 = keras.layers.Dense(256, activation = 'relu')
        self.d2 = keras.layers.Dense(100, activation = 'softmax')        
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.flat(x)
        x = self.d1(x)
        x = self.d2(x)
        return x 

In [56]:
model = SubclassModel()

In [57]:
model.compile(optimizer = 'adam', 
              loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 2s 2ms/step - loss: 3.6725 - accuracy: 0.1455
Epoch 2/5
1563/1563 [==============================] - 3s 2ms/step - loss: 2.9556 - accuracy: 0.2722
Epoch 3/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.6371 - accuracy: 0.3372
Epoch 4/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.4417 - accuracy: 0.3807
Epoch 5/5
1563/1563 [==============================] - 2s 2ms/step - loss: 2.2850 - accuracy: 0.4112
313/313 - 0s - loss: 2.5903 - accuracy: 0.3621


[2.5902581214904785, 0.3621000051498413]

### Gradient Tape 활용 
관련 분야: GAN, 강화학습 

In [58]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [59]:
import time
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 5.4639
Epoch 1: last batch loss = 2.4900
Epoch 2: last batch loss = 0.0083
Epoch 3: last batch loss = 0.0127
Epoch 4: last batch loss = 0.0755
It took 73.92261695861816 seconds


In [60]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 0s 1ms/step


0.1883